In [ ]:
! pip install --user keras

In [ ]:
! pip install --user tensorflow

In [ ]:
! pip install --user keras==1.2.2

In [1]:
import numpy as np
from keras.initializations import normal
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Input, Flatten
from keras.optimizers import SGD, Adam, Adamax
from keras.constraints import maxnorm
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import model_from_json
from keras.regularizers import l2, activity_l2, l1

from keras.callbacks import History 


seed = 13
np.random.seed(seed)


Using TensorFlow backend.


In [ ]:
! pip install --user import_ipynb
! pip install --user pandas

In [3]:
##### load training and testing dataset
from keras.utils.np_utils import to_categorical
import import_ipynb
import pandas as pds
def load_data(csv_path, shuffle=True):
    dat = pds.read_csv(csv_path)
    x_origin = dat.iloc[:, range(0, 6*20)].values
    y_origin = dat.loc[:, 'collision_system'].values
    if shuffle == True:
        randomize = np.arange(len(x_origin))
        np.random.shuffle(randomize)
        x = x_origin[randomize]
        y = y_origin[randomize]

    #x, y, randomize = regulize(x_origin, y_origin)
    return x, y


x_train, y_train = load_data('train_data.csv')
n_train = len(x_train)

x_test, y_test = load_data('testing_data.csv')
n_test = len(x_test)

height, width = 20, 6

print('shape=', x_train[0].shape)
print('shape=', y_train[0].shape)
y_train = to_categorical(y_train)
y_test= to_categorical(y_test)
print(y_train)

x_train = x_train.reshape(n_train, height, width, 1).astype('float32')
x_test = x_test.reshape(n_test, height, width, 1).astype('float32')

nb_classes = 2


shape= (120,)
shape= ()
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [16]:
!pip install --user tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 108kB/s  eta 0:00:01    |█▉                              | 23.4MB 12.7MB/s eta 0:00:31     |███████████████▏                | 195.3MB 70.3MB/s eta 0:00:04     |███████████████████▏            | 246.5MB 69.6MB/s eta 0:00:03     |███████████████████▍            | 249.8MB 69.6MB/s eta 0:00:03     |███████████████████████▊        | 305.6MB 60.0MB/s eta 0:00:02     |█████████████████████████▉      | 332.8MB 55.8MB/s eta 0:00:02     |██████████████████████████▎     | 339.1MB 55.8MB/s eta 0:00:02
     |████████████████████████████████| 512kB 64.4MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 61.2MB/s eta 0:00:01
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Consider adding this direc

In [26]:
conda remove tensorflow-gpu tensorflow tensorflow-base

Solving environment: failed

PackagesNotFoundError: The following packages are missing from the target environment:
  - tensorflow
  - tensorflow-base
  - tensorflow-gpu



Note: you may need to restart the kernel to use updated packages.


In [4]:
##### define the layer stacks
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
model = Sequential()

# used for LeakyReLU
leaky_alpha = 0.3



model.add(Convolution2D(18, 3,3, border_mode='same',
                        input_shape=(height, width, 1),
                        init='normal',
                        W_regularizer=l2(0.01))
                        )

model.add(BatchNormalization(mode=0, axis=3))
model.add(PReLU())
model.add(Dropout(0.2))

model.add(Convolution2D(32, 2, 2,
                border_mode='same',
                init='normal',
                W_regularizer=l2(0.01)))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='same', dim_ordering='default'))
model.add(BatchNormalization(mode=0, axis=3))
model.add(PReLU())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(0.01), init='normal'))
model.add(BatchNormalization(mode=0))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

adm = Adamax(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1.0E-6)


Instructions for updating:
non-resource variables are not supported in the long term

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:

model.compile(loss='categorical_crossentropy',
              optimizer=adm,
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="weights_filter.hdf5", verbose=1, save_best_only=True)

history = History()

model.fit(x_train, y_train,
          validation_split=0.1,
          nb_epoch=200, batch_size=64,
          callbacks=[checkpointer, history])

err, acc = model.evaluate(x_test, y_test, batch_size=1)
print('predict err=%s, acc=%s'%(err, acc))


model_json = model.to_json()
with open("model_filter.json", "w") as json_file:
    json_file.write(model_json)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 38678 samples, validate on 4298 samples
Epoch 1/200





38678/38678 [==============================] - 18s - loss: 3.0250 - acc: 0.5634 - val_loss: 2.3024 - val_acc: 0.6087
Epoch 2/200
38678/38678 [==============================] - 16s - loss: 2.0571 - acc: 0.5769 - val_loss: 1.6829 - val_acc: 0.6366
Epoch 3/200
38678/38678 [==============================] - 17s - loss: 1.6114 - acc: 0.5792 - val_loss: 1.3737 - val_acc: 0.6345
Epoch 4/200
38678/38678 [==============================] - 17s - loss: 1.3594 - acc: 0.5872 - val_loss: 1.1879 - val_acc: 0.6345
Epoch 5/200
38678/38678 [==============================] - 11s - loss: 1.2008 - acc: 0.5892 - val_loss: 1.0681 - val_acc: 0.6345
Epoch 6/200
38678/38678 [==============================] - 10s - loss: 1.0946 - acc: 0.5894 - val_loss: 0.9826 - val_acc: 0.6363
Epoch 7/200
38678/38678 [==============================] - 15s - loss: 1.0

In [21]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [6]:
# later ...
# load json and create model
with open('model_filter.json', 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("weights_filter.hdf5")

loaded_model.compile(loss='categorical_crossentropy',
              optimizer=adm,
              metrics=['accuracy'])

err, acc = loaded_model.evaluate(x_test, y_test, batch_size=1)
print('predict err=%s, acc=%s'%(err, acc))

hist_loss, val_loss, hist_acc, val_acc = history.history['loss'], history.history['val_loss'], history.history['acc'], history.history['val_acc']
np.savetxt('train_history_1_filter.txt', np.array([hist_loss, val_loss, hist_acc, val_acc]).T)


5931/5971 [============================>.] - ETA: 0spredict err=0.6466877739736132, acc=0.6343995980572769
